In [1]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests

In [2]:
def ex_tag_selenium(sid, page):
    driver = None
    try:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)

        url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}#&data=%2000:00:00&page={page}"
        driver.get(url)

        wait = WebDriverWait(driver, 20)
        a_tags = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#section_body a')))

        tag_set = set()
        for a in a_tags:
            href = a.get_attribute('href')
            if href and f"sid={sid}" in href and "article" in href:
                tag_set.add(href)

        tag_lst = list(tag_set)
        return tag_lst

    finally:
        if driver is not None:
            driver.quit()

In [3]:
def re_tag(sid, marker_url=None):
    re_lst = []
    marking = False
    
    for i in tqdm(range(100)):
        lst = ex_tag_selenium(sid, i + 1)
        re_lst.extend(lst)
        
        if marker_url and marker_url[sid - 100] in lst:
            print(f"Marker URL '{marker_url[sid - 100]}'가 중복되었습니다. 작업을 종료합니다.")
            marking = True
            break

    return re_lst, marking

In [4]:
def art_crawl(all_hrefs, sid, index, existing_data):
    art_dic = existing_data if existing_data else {}

    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans" \
                    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    url = all_hrefs[sid][index]
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0 "
                                                    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"
                                                    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")

    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)

    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)

    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)

    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str

    return art_dic

In [5]:
def crawl_news(marker_url=None):
    all_hrefs = {}
    sids = [i for i in range(100, 106)]
    artdic_lst = []

    for section in tqdm(sids):
        re_lst, marking = re_tag(section, marker_url=marker_url)
        all_hrefs[section] = re_lst

        for i in tqdm(range(len(re_lst))):
            existing_data = artdic_lst[-1] if artdic_lst else None
            art_dic = art_crawl(all_hrefs, section, i, existing_data)
            art_dic["section"] = section
            art_dic["url"] = re_lst[i]
            artdic_lst.append(art_dic)

            if marking and marker_url[section - 100] == art_dic["url"]:
                print(f"Marker URL '{marker_url[section - 100]}'가 중복되었습니다. 작업을 종료합니다.")
                break

    return artdic_lst

In [6]:
all_hrefs = {}
sids = [i for i in range(100, 106)]

for sid in sids:
    sid_data = re_tag(sid)
    

NameError: name 'all_hrefs' is not defined

In [ ]:
marker_urls_to_check = [all_hrefs[i][0] for i in range(100, 106)]
result_data = crawl_news(marker_url=marker_urls_to_check)